In [81]:
import numpy as np
import pandas as pd

In [98]:
raw_route_data = "D:/proj/ipa/schedule_info.csv"
intermediate_aggregated_delays = "D:/proj/ipa/aggregated_delays.pkl"
stations_lat_lon = "C:/proj/complex-data-analysis/data/stations_coordinates.csv"
final_data = "D:/proj/ipa/delays_on_stations.csv"

MINIMUM_DELAY = 15.0
DATE_FROM = "2018-06-01"
DATE_TO = "2018-06-07"

### Load data

In [99]:
routes = pd.read_csv(raw_route_data, header=None)
columns = ["schedule_id", "stop_number", "station_id", "arrival_time", "arrival_delay", "departure_time", "departure_delay"]
routes.columns = columns

### Fill missing data on start/end of route

In [100]:
# Missing delays
routes[["arrival_delay", "departure_delay"]] = routes[["arrival_delay", "departure_delay"]].fillna(0.0)

# Missing departure times
routes["departure_time"] = routes["departure_time"].fillna(routes["arrival_time"])

In [101]:
routes.tail()

,schedule_id,stop_number,station_id,arrival_time,arrival_delay,departure_time,departure_delay
16162901,53564670,32,1570,2018-12-08 00:34:30,0.0,2018-12-08 00:35:00,0.0
16162902,53564670,33,1571,2018-12-08 00:38:00,0.0,2018-12-08 00:38:30,0.0
16162903,53564670,34,1572,2018-12-08 00:40:30,0.0,2018-12-08 00:41:00,0.0
16162904,53564670,35,1573,2018-12-08 00:43:30,0.0,2018-12-08 00:44:00,0.0
16162905,53564670,36,173,2018-12-08 00:49:00,0.0,2018-12-08 00:49:00,0.0


### Prepare data for rolling aggregations

In [102]:
# Here, we consider only departure delays
is_delayed = routes["departure_delay"] > MINIMUM_DELAY

# Subset one month
one_month = (routes["departure_time"] > DATE_FROM) & (routes["departure_time"] <= DATE_TO)

In [103]:
# Filter delayed departures
delayed_routes = routes[is_delayed & one_month]
delayed_routes = delayed_routes[["station_id", "departure_time", "departure_delay"]]

# Convert departure time to datetime series
delayed_routes["departure_time"] = pd.to_datetime(delayed_routes["departure_time"])

# Set departure_time as index
delayed_routes = delayed_routes.set_index("departure_time")

In [104]:
delayed_routes.head()

,station_id,departure_delay
departure_time,,
2018-06-01 23:09:00,47,28.0
2018-06-01 23:14:30,251,29.0
2018-06-01 23:22:00,250,29.0
2018-06-01 23:25:00,249,28.0
2018-06-01 23:28:30,248,27.0


### Aggregate delays in 15-minute timeframe
Done by grouping by `departure_time` using `pd.Grouper` with 15 minutes frequency.

In [105]:
accumulated_delays = delayed_routes.groupby(
    [
        pd.Grouper(level="departure_time", freq="15min"), 
        "station_id"
    ]
).sum()


In [106]:
accumulated_delays.head(n=10)

departure_delay
departure_time      station_id                 
2018-06-01 00:00:00 243                    22.0
2018-06-01 00:30:00 238                    70.0
2018-06-01 00:45:00 204                    25.0
2018-06-01 01:00:00 107                    75.0
2018-06-01 01:15:00 108                    71.0
2018-06-01 01:30:00 43                     53.0
                    109                    62.0
                    118                    21.0
2018-06-01 01:45:00 42                     50.0
2018-06-01 02:00:00 41                     43.0

In [107]:
accumulated_delays.shape

(13337, 1)

In [108]:
accumulated_delays.to_pickle(intermediate_aggregated_delays)

### Join stations with lat/lon

In [109]:
stations = pd.read_csv(stations_lat_lon)
stations.columns = ["station_id", "station", "latitude", "longitude"]
stations = stations.set_index("station_id")

In [110]:
stations.head()

,station,latitude,longitude
station_id,,,
181,Aleksandrów Kujawski,52.875865,18.696391
1321,Andrychów,49.855792,19.353358
1322,Andrychów Górnica,49.858467,19.323956
1644,Andrzejówka,51.173162,15.521410
1000,Anieliny,53.129123,17.486455


In [111]:
accumulated_delays = accumulated_delays.reset_index()
station_delays = accumulated_delays.merge(stations, on="station_id")

In [112]:
station_delays.head()

,departure_time,station_id,departure_delay,station,latitude,longitude
0,2018-06-01 00:00:00,243,22.0,Dulowa,50.140444,19.52075
1,2018-06-02 10:45:00,243,35.0,Dulowa,50.140444,19.52075
2,2018-06-02 14:30:00,243,19.0,Dulowa,50.140444,19.52075
3,2018-06-02 19:30:00,243,24.0,Dulowa,50.140444,19.52075
4,2018-06-03 13:45:00,243,56.0,Dulowa,50.140444,19.52075


In [113]:
station_delays[["departure_time", "departure_delay", "latitude", "longitude"]].to_csv(final_data)